In [1]:
import os

from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

import numpy as np
import torch
import torch.nn as nn

from models.pointnet import PointNet, TNet
from models.modules import SpecialEuclideanGeodesicLoss

from runners.experiment import run_epoch

from torchinfo import summary

def _R(theta, phi, psi, N):
    return (torch.tensor([
        [np.cos(theta), -np.sin(theta), 0],
        [np.sin(theta), np.cos(theta), 0],
        [0, 0, 1]
    ]) @ torch.tensor([
        [np.cos(phi), 0, np.sin(phi)],
        [0, 1, 0],
        [-np.sin(phi), 0, np.cos(phi)]
    ]) @ torch.tensor([
        [1, 0, 0],
        [0, np.cos(psi), -np.sin(psi)],
        [0, np.sin(psi), np.cos(psi)]
    ])).unsqueeze(0).float().repeat(N, 1, 1)

def _T(x, y, z, N):
    return torch.tensor([[x], [y], [z]]).unsqueeze(0).float().repeat(N, 1, 1)

def RT(angles, translations, N):
    r = _R(*angles, N)
    t = _T(*translations, N)
    return torch.cat((r, t), dim=-1)


In [2]:
SHAPE = 10, 3
B = 2

LR = 0.0001
WD = 0.0001

model = PointNet()
criterion = SpecialEuclideanGeodesicLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WD)

x = torch.ones((B, *SHAPE))

angles = 0.1, 0.1, 0.1
translations = 0.1, 0.2, 0.3

y = RT(angles, translations, B)

for _ in range(1000):
    optimizer.zero_grad()
    outputs = model(x)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()

model(x)


torch.Size([2, 3, 3])
tensor(2.3726, grad_fn=<MeanBackward0>) tensor(0.0455, grad_fn=<MseLossBackward0>)
torch.Size([2, 3, 3])
tensor(2.3629, grad_fn=<MeanBackward0>) tensor(0.0454, grad_fn=<MseLossBackward0>)
torch.Size([2, 3, 3])
tensor(2.3546, grad_fn=<MeanBackward0>) tensor(0.0454, grad_fn=<MseLossBackward0>)
torch.Size([2, 3, 3])
tensor(2.3456, grad_fn=<MeanBackward0>) tensor(0.0454, grad_fn=<MseLossBackward0>)
torch.Size([2, 3, 3])
tensor(2.3362, grad_fn=<MeanBackward0>) tensor(0.0453, grad_fn=<MseLossBackward0>)
torch.Size([2, 3, 3])
tensor(2.3267, grad_fn=<MeanBackward0>) tensor(0.0453, grad_fn=<MseLossBackward0>)
torch.Size([2, 3, 3])
tensor(2.3169, grad_fn=<MeanBackward0>) tensor(0.0453, grad_fn=<MseLossBackward0>)
torch.Size([2, 3, 3])
tensor(2.3071, grad_fn=<MeanBackward0>) tensor(0.0452, grad_fn=<MseLossBackward0>)
torch.Size([2, 3, 3])
tensor(2.2971, grad_fn=<MeanBackward0>) tensor(0.0452, grad_fn=<MseLossBackward0>)
torch.Size([2, 3, 3])
tensor(2.2869, grad_fn=<MeanBackw

_LinAlgError: linalg.svd: (Batch element 0): The algorithm failed to converge because the input matrix contained non-finite values.

In [ ]:
print(y)


tensor([[[ 0.9900, -0.0894,  0.1088,  0.1000],
         [ 0.0993,  0.9910, -0.0894,  0.2000],
         [-0.0998,  0.0993,  0.9900,  0.3000]],

        [[ 0.9900, -0.0894,  0.1088,  0.1000],
         [ 0.0993,  0.9910, -0.0894,  0.2000],
         [-0.0998,  0.0993,  0.9900,  0.3000]]])
